## First tests in getting data from Jira

In [5]:
from jira import JIRA
from osbot_jira.api.jira_server.API_Jira import API_Jira

api_jira = API_Jira()
api_jira._jira = JIRA(server='https://ubuntu-policy.atlassian.net/')

from IPython.display        import display_html, HTML, Javascript,display
from osbot_aws.apis.Lambda import Lambda
def maps_create(code):
    aws_lambda = Lambda('osbot_browser.lambdas.lambda_browser')
    params = ["maps", "exec_js"]
    params.extend(code.split(' '))

    payload = {"params": params,
               'data': {}}
    png_data = aws_lambda.invoke(payload)
    show_png(png_data)

#maps_create(map)
def show_png(png_data,height=200):
    html = '<img style="width:100%;height:800px;border:1px solid black" align="left" src="data:image/png;base64,{1}"/>'.format(height,png_data)
    display_html(html, raw=True)    
    

In [3]:
issue = api_jira.issue('Map-1')


In [6]:
map_code = """
add("wizard in action", 1, 1)

add("cast a spell", 1, 2) 

SyntaxError: EOF while scanning triple-quoted string literal (<ipython-input-6-d2442e548fa6>, line 4)